In [1]:
class JumpingHeadTM:
    def __init__(self, tape, transitions, start_state, accept_state, reject_state):
        self.tape = list(tape)
        self.transitions = transitions
        self.state = start_state
        self.head = 0
        self.accept = accept_state
        self.reject = reject_state

    def step(self):
        symbol = self.tape[self.head] if self.head < len(self.tape) else '_'
        key = (self.state, symbol)

        if key in self.transitions:
            new_symbol, direction, next_state = self.transitions[key]

            if self.head >= len(self.tape):
                self.tape.extend(['_'] * (self.head - len(self.tape) + 1))

            print(f"Step: {self.state} --[{symbol}/{new_symbol},{direction}]--> {next_state} | Head: {self.head} | Tape: {''.join(self.tape)}")

            self.tape[self.head] = new_symbol
            self.state = next_state

            if direction == 'R':
                self.head += 2
            elif direction == 'L':
                self.head = max(0, self.head - 2)
            # 'N' = não move
            return True
        else:
            self.state = self.reject
            return False

    def run(self, max_steps=1000):
        steps = 0
        while self.state not in {self.accept, self.reject} and steps < max_steps:
            if not self.step():
                break
            steps += 1

        print(f"\nFinal: estado={self.state}, head={self.head}, tape={''.join(self.tape)}")
        return (self.state == self.accept), ''.join(self.tape), steps


Verificar se a cadeia binária contém quantidade par de 1s

In [2]:
t_jump_paridade_1s = {
    # Estado inicial: paridade atual = par
    ('q0', '1'): ('X', 'R', 'q1a'),
    ('q0', '0'): ('0', 'R', 'q0a'),
    ('q0', '_'): ('_', 'N', 'q_acc'),

    # Continuação da paridade par
    ('q0a', '1'): ('X', 'R', 'q1b'),
    ('q0a', '0'): ('0', 'R', 'q0b'),
    ('q0a', '_'): ('_', 'N', 'q_acc'),

    ('q0b', '1'): ('X', 'R', 'q1'),
    ('q0b', '0'): ('0', 'R', 'q0'),
    ('q0b', '_'): ('_', 'N', 'q_acc'),

    # Paridade ímpar (após ler um 1)
    ('q1', '1'): ('X', 'R', 'q0a'),
    ('q1', '0'): ('0', 'R', 'q1a'),
    ('q1', '_'): ('_', 'N', 'q_rej'),

    ('q1a', '1'): ('X', 'R', 'q0b'),
    ('q1a', '0'): ('0', 'R', 'q1b'),
    ('q1a', '_'): ('_', 'N', 'q_rej'),

    ('q1b', '1'): ('X', 'R', 'q0'),
    ('q1b', '0'): ('0', 'R', 'q1'),
    ('q1b', '_'): ('_', 'N', 'q_rej'),
}


In [3]:
# Se finaliza em q2 vindo de q1, rejeita
final_rejecting = ['q1', 'q1a', 'q1b']
for state in final_rejecting:
    t_jump_paridade_1s[('q2', state)] = ('_', 'N', 'q_rej')

In [4]:
tape = '0_1_1_0'
tm = JumpingHeadTM(tape, t_jump_paridade_1s, 'q0', 'q_acc', 'q_rej')
print(tm.run())

Step: q0 --[0/0,R]--> q0a | Head: 0 | Tape: 0_1_1_0
Step: q0a --[1/X,R]--> q1b | Head: 2 | Tape: 0_1_1_0
Step: q1b --[1/X,R]--> q0 | Head: 4 | Tape: 0_X_1_0
Step: q0 --[0/0,R]--> q0a | Head: 6 | Tape: 0_X_X_0
Step: q0a --[_/_,N]--> q_acc | Head: 8 | Tape: 0_X_X_0__

Final: estado=q_acc, head=8, tape=0_X_X_0__
(True, '0_X_X_0__', 5)


In [5]:
tm_err = JumpingHeadTM('1_0_0', t_jump_paridade_1s, 'q0', 'q_acc', 'q_rej')
print(tm_err.run())


Step: q0 --[1/X,R]--> q1a | Head: 0 | Tape: 1_0_0
Step: q1a --[0/0,R]--> q1b | Head: 2 | Tape: X_0_0
Step: q1b --[0/0,R]--> q1 | Head: 4 | Tape: X_0_0
Step: q1 --[_/_,N]--> q_rej | Head: 6 | Tape: X_0_0__

Final: estado=q_rej, head=6, tape=X_0_0__
(False, 'X_0_0__', 4)


Reconhecer cadeias com o padrão a_b_a_b... (intercalado)

In [6]:
t_jump_intercalado = {
    # Início: espera 'a'
    ('q0', 'a'): ('X', 'R', 'qa1'),
    ('q0', '_'): ('_', 'N', 'q_acc'),

    # Após 'a', espera 'b'
    ('qa1', 'b'): ('Y', 'R', 'qa2'),
    ('qa1', 'a'): ('a', 'N', 'q_rej'),
    ('qa1', '_'): ('_', 'N', 'q_rej'),
    ('qa1', 'X'): ('X', 'R', 'qa1'),
    ('qa1', 'Y'): ('Y', 'R', 'qa1'),

    # Após marcar b, transição para próximo ciclo
    ('qa2', 'a'): ('X', 'R', 'qa3'),
    ('qa2', '_'): ('_', 'N', 'q_acc'),
    ('qa2', 'b'): ('b', 'N', 'q_rej'),
    ('qa2', 'X'): ('X', 'R', 'qa2'),
    ('qa2', 'Y'): ('Y', 'R', 'qa2'),

    # qa3 espera b de novo
    ('qa3', 'b'): ('Y', 'R', 'qa2'),
    ('qa3', 'a'): ('a', 'N', 'q_rej'),
    ('qa3', '_'): ('_', 'N', 'q_acc'),
    ('qa3', 'X'): ('X', 'R', 'qa3'),
    ('qa3', 'Y'): ('Y', 'R', 'qa3'),
}


In [7]:
tm = JumpingHeadTM('a_b_a_b_', t_jump_intercalado, 'q0', 'q_acc', 'q_rej')
print(tm.run())


Step: q0 --[a/X,R]--> qa1 | Head: 0 | Tape: a_b_a_b_
Step: qa1 --[b/Y,R]--> qa2 | Head: 2 | Tape: X_b_a_b_
Step: qa2 --[a/X,R]--> qa3 | Head: 4 | Tape: X_Y_a_b_
Step: qa3 --[b/Y,R]--> qa2 | Head: 6 | Tape: X_Y_X_b_
Step: qa2 --[_/_,N]--> q_acc | Head: 8 | Tape: X_Y_X_Y__

Final: estado=q_acc, head=8, tape=X_Y_X_Y__
(True, 'X_Y_X_Y__', 5)


In [8]:
tm2 = JumpingHeadTM('a_b_b_', t_jump_intercalado, 'q0', 'q_acc', 'q_rej')
print(tm2.run())

Step: q0 --[a/X,R]--> qa1 | Head: 0 | Tape: a_b_b_
Step: qa1 --[b/Y,R]--> qa2 | Head: 2 | Tape: X_b_b_
Step: qa2 --[b/b,N]--> q_rej | Head: 4 | Tape: X_Y_b_

Final: estado=q_rej, head=4, tape=X_Y_b_
(False, 'X_Y_b_', 3)
